In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/MyDrive/Deep_Learning_course_VT/DL project/covid_xrays_model"
%pip install -e .

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

# from covid_xrays_model import config

from fastai.vision import DatasetType
import pandas as pd
import joblib
from covid_xrays_model.processing.data_management import load_dataset
from covid_xrays_model.predict import predict_dataset
from covid_xrays_model.train_pipeline import run_training_sample

import warnings
warnings.filterwarnings('ignore')

# Set your data path here

In [ ]:
os.environ['DATA_DIR'] = '/home/doaa/Git/covid_xrays/data'
# os.environ['DATA_DIR'] = "/content/drive/MyDrive/Deep_Learning_course_VT/DL project/data"

# FASTAI classification model

In [ ]:
data = load_dataset(image_size=224, sample_size=5000)
data

In [ ]:
data.show_batch(rows=3, figsize=(10,8), ds_type=DatasetType.Train)


In [ ]:
# data.train_ds
data.classes, len(data.train_ds), len(data.valid_ds), data.c

In [ ]:
# data.test_ds[0][0]

# Run training

In [ ]:
sample_size = 5001   # number of training images per class
percent_gan_images = 10  # 25%
confusion_matrix_filename = f'conf_mat_{sample_size}_{percent_gan_images}'
print(confusion_matrix_filename)

In [ ]:
run_training_sample(n_cycles=10,
                    sample_size=sample_size, 
                    percent_gan_images=percent_gan_images,   # Percent of GAN generated COVID images to add
                    confusion_matrix_filename=confusion_matrix_filename,
                    with_oversampling=True,     # always use True
                    with_weighted_loss=True)   # always use True

### Load training Confusion Matrix

In [ ]:
train_mat = joblib.load(f'train_{confusion_matrix_filename}.pkl')
# sum diagonal / all data_size *100
accuracy = np.trace(train_mat) / train_mat.sum() * 100

print(f'Accuracy: {accuracy:.2f}%')
print(f'Confusion Matrix:\n {train_mat}')

### For Validation confusion Matrix

In [ ]:
valid_mat = joblib.load(f'valid_{confusion_matrix_filename}.pkl')
# sum diagonal / all data_size *100
accuracy = np.trace(valid_mat) / valid_mat.sum() * 100

print(f'Accuracy: {accuracy:.2f}%')
print(f'Confusion Matrix:\n {valid_mat}')

# Run evaluation (model has to be already trained) - slow on CPU

In [ ]:
# Evaluate test set (ds_type='test')
# Will load the correct saved model based on the given parameters

test_conf_mat, test_accuracy = predict_dataset(ds_type='test', 
                                               cpu=False,   # set to False if running on GPUs
                                               sample_size=sample_size,  
                                               percent_gan_images=percent_gan_images,
                                               with_oversampling=True,   # always use True
                                               with_weighted_loss=True,  # always use True
                                               confusion_matrix_filename=confusion_matrix_filename
                                              )

In [ ]:
test_conf_mat = joblib.load(f'test_{confusion_matrix_filename}.pkl')
# sum diagonal / all data_size *100
test_accuracy = np.trace(test_conf_mat) / test_conf_mat.sum() * 100

print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'Test Confusion Matrix:\n {test_conf_mat}') 

# Play with the model

In [ ]:
learn.predict(data.test_ds[0][0])